In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from category_encoders import BinaryEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from joblib import dump,load
import graphviz
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score

In [20]:
data = pd.read_csv('student_addiction_dataset_test.csv')
data.head()

,Experimentation,Academic_Performance_Decline,Social_Isolation,Financial_Issues,Physical_Mental_Health_Problems,Legal_Consequences,Relationship_Strain,Risk_Taking_Behavior,Withdrawal_Symptoms,Denial_and_Resistance_to_Treatment,Addiction_Class
0,Yes,Yes,Yes,No,Yes,No,No,No,Yes,No,No
1,No,No,Yes,No,No,Yes,No,Yes,No,No,No
2,No,No,No,Yes,No,Yes,No,No,Yes,No,No
3,Yes,No,Yes,No,Yes,Yes,No,Yes,No,Yes,Yes
4,No,No,No,No,Yes,No,No,No,No,Yes,No


In [21]:
n_data = data.dropna()
n_data

,Experimentation,Academic_Performance_Decline,Social_Isolation,Financial_Issues,Physical_Mental_Health_Problems,Legal_Consequences,Relationship_Strain,Risk_Taking_Behavior,Withdrawal_Symptoms,Denial_and_Resistance_to_Treatment,Addiction_Class
0,Yes,Yes,Yes,No,Yes,No,No,No,Yes,No,No
1,No,No,Yes,No,No,Yes,No,Yes,No,No,No
2,No,No,No,Yes,No,Yes,No,No,Yes,No,No
3,Yes,No,Yes,No,Yes,Yes,No,Yes,No,Yes,Yes
4,No,No,No,No,Yes,No,No,No,No,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
12736,Yes,Yes,No,Yes,Yes,No,Yes,No,No,Yes,Yes
12737,Yes,No,No,No,No,No,No,No,Yes,Yes,No
12738,Yes,No,No,Yes,Yes,No,Yes,No,Yes,No,No
12741,No,No,Yes,No,No,Yes,Yes,No,No,No,Yes


In [22]:
x = n_data.drop(columns=['Addiction_Class'])
y = n_data['Addiction_Class']

In [23]:
categorical_features = [
    'Experimentation',
    'Academic_Performance_Decline',
    'Social_Isolation',
    'Financial_Issues',
    'Physical_Mental_Health_Problems',
    'Legal_Consequences',
    'Relationship_Strain',
    'Risk_Taking_Behavior',
    'Withdrawal_Symptoms',
    'Denial_and_Resistance_to_Treatment'
]
encoder = BinaryEncoder()
x_encoded = encoder.fit_transform(x[categorical_features])
x = x.drop(columns=categorical_features)
x = pd.concat([x, x_encoded], axis=1)

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)

In [25]:
smote = SMOTE(random_state=45)
x_train, y_train = smote.fit_resample(x_train, y_train)

In [26]:
dcs = DecisionTreeClassifier()
dcs.fit(x_train, y_train)

export_graphviz(dcs, out_file="tree.dot", filled=True, feature_names=x_train.columns)

with open("tree.dot") as f:
    dot_graph = f.read()

graphviz.Source(dot_graph)

dump(dcs,"decision_tree_model.joblib")

['decision_tree_model.joblib']

In [27]:
prediction = dcs.predict(x_test)

accuracy = accuracy_score(y_test, prediction)
precision = precision_score(y_test, prediction, average='micro')

print(f"Accuracy: {accuracy * 100}%\nPrecision: {precision * 100}%")

Accuracy: 63.75731772219265%
Precision: 63.75731772219265%
